In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
def train_test_split(data,test_size):
    
    if isinstance(test_size,float):
        test_size = round(test_size*len(data))

    indices = data.index.tolist()
    test_indices = random.sample(population=indices,k=test_size) #Sample function operate on list

    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    
    return train_data, test_data

In [3]:
def check_purity(data):
    
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [4]:
def classify_data(data):
    
    label_column = data[:,-1]
    unique_classes, count_unique_classes = np.unique(label_column, return_counts=True)
    index = count_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [5]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_thrashold = 15
    
    for column in df.columns:
        unique_values = df[column].unique()
        example_value = unique_values[0]
        
        if (isinstance(example_value,str)) or (len(unique_values) <= n_unique_values_thrashold):
            feature_types.append("Catagorical")
        else:
            feature_types.append("Continuous")
    
    return feature_types

In [6]:
def get_potential_splits(data, random_subspace):
    
    potential_splits = {}
    n = np.size(data,axis=1)
    column_indicies = list(range(n-1))
    
    if random_subspace <= len(column_indicies):
        column_indicies = random.sample(population=column_indicies,k=random_subspace)
    
    for column_index in column_indicies:
        unique_values = np.unique(data[:,column_index])
        
        potential_splits[column_index] = unique_values
                
    return potential_splits

In [7]:
def calculate_entropy(data):
    
    label_column = data[:,-1]
    _,counts = np.unique(label_column,return_counts=True)
    probablities = counts/(np.sum(counts))
    entropy = sum(probablities * (-np.log2(probablities)))
    
    return entropy

def calculate_overall_entropy(data_below,data_above):
    
    n_data_points = len(data_below) + len(data_above)
    p_data_below = len(data_below)/n_data_points
    p_data_above = len(data_above)/n_data_points
    overall_entropy = (p_data_below * calculate_entropy(data_below)) + (p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [8]:
def determine_best_split(data, potential_splits):
    overall_entropy = 999
    
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below,data_above = data_split(data,column_index,value)
            current_overall_entropy = calculate_overall_entropy(data_below,data_above)
            
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
                
    return best_split_column,best_split_value

In [9]:
def data_split(data,split_column,split_value):
    
    split_column_values = data[:,split_column]
    
    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == 'Continuous':
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values > split_value]
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below,data_above

In [10]:
def decision_tree_algorithm(tf,counter=0,min_samples=2,max_depth=5,random_subspace=None):
    
    #data preperation
    if(counter == 0):
        global COLUUMN_HEADER, FEATURE_TYPES
        COLUUMN_HEADER = tf.columns
        FEATURE_TYPES = determine_type_of_feature(tf)
        data = tf.values
    else:
        data = tf
        
    #base_case
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    #recursive_part
    else:
        counter += 1
        
        #helper functions
        potential_splits = get_potential_splits(data, random_subspace)
        best_split_column,best_split_value = determine_best_split(data,potential_splits)
        data_below, data_above = data_split(data, best_split_column, best_split_value)
        
        #check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        #instate sub tree
        feature_column = COLUUMN_HEADER[best_split_column]
        type_of_column = FEATURE_TYPES[best_split_column]
        if type_of_column == "Continuous":
            question = "{} <= {}".format(feature_column,best_split_value)
        else:
            question = "{} = {}".format(feature_column,best_split_value)
        sub_tree = {question: []}
        
        #find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below,counter,min_samples,max_depth,random_subspace)
        no_answer = decision_tree_algorithm(data_above,counter,min_samples,max_depth,random_subspace)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [11]:
def classify_example(example,tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    #ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    #base_Case
    if not isinstance(answer,dict):
        return answer
    #rescursive_part
    else:
        residual_tree = answer
        return classify_example(example,residual_tree)

In [12]:
def bootstrapping(train_df, n_bootstrap):
    bootstrap_indices = np.random.randint(0,len(train_df),n_bootstrap)
    df_bootstrap = train_df.iloc[bootstrap_indices]

    return df_bootstrap

In [13]:
def random_forest_algorithm(tf,n_trees,n_bootstrap,n_features, dt_max_depth):
    forest = []
    
    for i in range(n_trees):
        df_bootstrap = bootstrapping(tf,n_bootstrap)
        tree = decision_tree_algorithm(df_bootstrap,max_depth=dt_max_depth,random_subspace=n_features)
        forest.append(tree)
    
    return forest

In [14]:
def random_forest_predictions(test,forest):
    
    df_predictions = {}
    for i in range(len(forest)):
        column_name = "Tree_{}".format(i)
        predictions = []
        for j in range(len(test)):
            prediction = classify_example(test.iloc[j], tree=forest[i])
            predictions.append(prediction)
        df_predictions[column_name] = predictions
    df_predictions = pd.DataFrame(df_predictions)
    
    random_forest_predictions = df_predictions.mode(axis=1)[0]
    return random_forest_predictions

In [15]:
def calculate_accuracy(df,predictions):
    
    df = df.reset_index(drop=True)
    df['correct_classification'] = predictions == df['Species']
    accuracy = df.correct_classification.mean()
    
    return accuracy